In [16]:
import subprocess
import os


def compile_and_run_cpp(source_file, output_file="output.txt", executable_name="a.exe", iterations=20):
    # Compile the C++ file
    compile_command = ["g++", source_file, "-o", executable_name]
    
    try:
        subprocess.run(compile_command, check=True)
        print(f"Compilation successful: {executable_name}")
    except subprocess.CalledProcessError:
        print("Compilation failed!")
        return
    
    # Run the executable 20 times and capture output in append mode
    try:
                # Check if the output file exists, and delete it if it does
        if os.path.exists(output_file):
            os.remove(output_file)
            print(f"Pre-existing file '{output_file}' deleted.")

        with open(output_file, "a") as out_file:
            for i in range(1, iterations + 1):
                # out_file.write(f"\n--- Execution {i} ---\n")
                subprocess.run([executable_name], stdout=out_file, stderr=subprocess.STDOUT, check=True)
        print(f"All executions completed. Output saved to '{output_file}'.")
    except subprocess.CalledProcessError:
        print("Execution failed!")

        return
    except FileNotFoundError:
        print(f"File not found: {executable_name}")
        return
    except Exception as e:
        print(f"An error occurred: {e}")
        return
    



In [17]:
cpp_file = "P1T3.cpp"
# cpp_file = "P1T3_Using2LSFRHashing.cpp"
# cpp_file = "P1T3_Using2stageHashing.cpp"
# cpp_file = "PredefinedHash.cpp"
output_file_name = "CompleteGraph"
# output_file_name = "AugmentedChain"
# output_file_name = "Erdos-Renyi"
output_file = f"{output_file_name}.txt"
executable_name = "a.exe"
iterations = 20
compile_and_run_cpp(cpp_file, output_file, executable_name, iterations)

Compilation successful: a.exe
All executions completed. Output saved to 'CompleteGraph.txt'.


In [18]:
import pandas as pd
import re

rows = []
with open(f"d:\\Semester 8\\CS6170_Randomized Algorithms\\Term Paper\\{output_file_name}.txt", "r") as f:
    lines = f.readlines()

i = 0
while i < len(lines):
    line = lines[i].strip()
    # We start a new block when we see a separator line.
    if line.startswith("---------------------------------------------------"):
        iteration_count = 0
        i += 1
        # Count the occurrences of [DerandomizedColorCoding::has_colorful_path] before reaching the Complete Graph block.
        while i < len(lines):
            curr_line = lines[i].strip()
            if curr_line.startswith(("Complete Graph:", "Layered Graph:", "Augmented Chain Graph:","Erdos-Renyi Graph:")):
                break
            if "[DerandomizedColorCoding::has_colorful_path]" in curr_line:
                iteration_count += 1
            i += 1

        # Skip the "Complete Graph:" header.
        if i < len(lines) and lines[i].strip().startswith(("Complete Graph:","Layered Graph:", "Augmented Chain Graph:","Erdos-Renyi Graph:")):
            i += 1
        
        # Next line with vertices, edges, and path length.
        if i < len(lines):
            vg_line = lines[i].strip()
            m = re.search(r"Vertices:\s*(\d+),\s*Edges:\s*(\d+),\s*Path length:\s*(\d+)", vg_line)
            if m:
                vertices = int(m.group(1))
                edges = int(m.group(2))
                pathlength = int(m.group(3))
            else:
                vertices = edges = pathlength = None
            i += 1
        else:
            break
        
        # Next line with "Contains colorful path" information.
        if i < len(lines):
            cp_line = lines[i].strip()
            m = re.search(r"Contains colorful path:\s*(\w+)", cp_line, re.IGNORECASE)
            if m:
                contains_colorful_path = m.group(1).lower() == "true"
            else:
                contains_colorful_path = None
            i += 1
        else:
            break
        
        # Next line with the final "Time taken" for the Complete Graph block.
        if i < len(lines):
            time_line = lines[i].strip()
            m = re.search(r"Time taken:\s*(\d+)\s*microseconds", time_line)
            if m:
                time_taken = int(m.group(1))
            else:
                time_taken = None
            i += 1
        else:
            break

        rows.append({
            "vertices": vertices,
            "Edges": edges,
            "pathlength": pathlength,
            "contains colourful path": contains_colorful_path,
            "time taken": time_taken,
            "iteration": iteration_count
        })
    else:
        i += 1

df = pd.DataFrame(rows)
print(df)
#droppin column contains colourful path 
df.drop(columns=['contains colourful path'], inplace=True)
df.to_csv(f"{output_file_name}.csv", index=False)





     vertices   Edges  pathlength  contains colourful path  time taken  \
0          10      90           8                     True         971   
1          20     380           8                     True        2464   
2          30     870           8                     True        4995   
3          40    1560           8                     True        7283   
4          50    2450           8                     True       10514   
..        ...     ...         ...                      ...         ...   
355       300   89700           8                     True      262673   
356       350  122150           8                     True      353166   
357       400  159600           8                     True      456380   
358       450  202050           8                     True      565137   
359       500  249500           8                     True      698591   

     iteration  
0            1  
1            1  
2            1  
3            1  
4            1  
..       

In [19]:

#taking all the graphs having same vertices and taking the mean of the number of edges, time taken , path length and iteration
df1 = df.groupby(['vertices']).agg({'Edges': 'mean', 'pathlength': 'mean', 'time taken': 'mean', 'iteration': 'mean'}).reset_index()
# df.groupby(['vertices'])
# adding the actual value of iteration to the dataframe
# df1['iteration'] = df1['iteration']*iterations
df1


,vertices,Edges,pathlength,time taken,iteration
0,10,90.0,8.0,964.05,1.0
1,20,380.0,8.0,2485.50,1.0
2,30,870.0,8.0,4613.75,1.0
3,40,1560.0,8.0,7215.95,1.0
4,50,2450.0,8.0,10523.60,1.0
5,60,3540.0,8.0,14489.15,1.0
6,70,4830.0,8.0,18755.00,1.0
7,80,6320.0,8.0,23413.55,1.0
8,90,8010.0,8.0,28831.20,1.0
9,100,9900.0,8.0,34782.75,1.0


In [20]:
df1.to_csv(f"{output_file_name}1.csv", index=False)